In [50]:
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.signal as signal
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import seaborn as sns
import warnings
import sys, os

warnings.filterwarnings('ignore')
random_state = 6
np.random.seed(random_state)

In [51]:
# filename = 'seismogram_data_62.csv'
filename = 'seismogram_data_62_new.csv'
df = pd.read_csv(filename)

In [52]:
df.describe()

,mfccs_0,mfccs_1,mfccs_2,mfccs_3,mfccs_4,mfccs_5,mfccs_6,mfccs_7,mfccs_8,mfccs_9,...,centiroid,max_amplitude,mean_amplitude,moment,variation,skew,var,autocr,kurto,target
count,23496.000000,23496.000000,23496.000000,23496.000000,23496.000000,23496.000000,23496.000000,23496.000000,23496.000000,23496.000000,...,23496.000000,2.349600e+04,2.349600e+04,23496.0,23496.000000,23496.000000,2.349600e+04,2.349600e+04,23496.000000,23496.000000
mean,842.505003,73.359982,-0.894606,52.395407,2.601928,34.513327,7.728752,22.589093,12.131532,15.490778,...,959.045374,2.471252e+08,3.094769e+05,0.0,18.969707,-0.032753,5.274213e+11,1.096999e+17,7.173602,0.448332
std,189.732512,38.111069,38.966119,19.352436,24.226235,12.969548,15.588723,9.172668,10.312101,6.988078,...,1024.497248,1.740120e+09,4.249657e+06,0.0,3913.459504,1.590172,2.553565e+13,3.697947e+18,211.110656,0.497334
min,480.726115,-87.232311,-198.422667,-32.555981,-88.160009,-41.837174,-48.987699,-26.721758,-35.454891,-23.973469,...,3.588867,4.490561e+03,4.140245e+01,0.0,-427348.312500,-148.525772,0.000000e+00,1.098498e+06,-3.000000,0.000000
25%,701.650737,45.531983,-28.158299,39.503115,-15.940220,25.904370,-3.266232,16.958161,6.091282,11.349311,...,199.003485,5.968881e+05,1.863994e+03,0.0,-0.332406,-0.082917,3.750436e+04,1.124291e+10,-0.069240,0.000000
50%,825.624647,74.008466,2.385506,53.200929,3.833513,34.087116,9.972833,22.015254,14.141603,15.476138,...,546.485180,1.234899e+06,3.855819e+03,0.0,0.196938,-0.002934,1.565367e+05,4.730443e+10,0.186421,0.000000
75%,959.726289,99.577733,28.229987,65.021862,22.487635,42.324745,20.381357,27.575977,19.567520,19.274661,...,1442.009632,8.348212e+06,2.115913e+04,0.0,0.891293,0.063645,3.800722e+06,1.896499e+12,2.320519,1.000000
max,2065.265843,233.043814,112.815778,132.919902,64.457973,118.624193,58.102843,71.468831,53.939397,65.529958,...,6894.203309,7.142860e+10,5.437551e+08,0.0,287691.593750,67.194862,3.695033e+15,4.833089e+20,27424.243210,1.000000


In [53]:
Y = df['target']
X = df.drop(['target'], axis=1)

print(np.shape(X), np.shape(Y))

X, Y = shuffle(X, Y, random_state = random_state)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state=random_state)
mms = StandardScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.fit_transform(X_test)

(23496, 61) (23496,)


## LightGBM

In [56]:
import lightgbm as lgb
lgbm_model = lgb.LGBMClassifier()
lgbm_model.fit(X_train, Y_train)

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0.0, n_estimators=10, n_jobs=-1,
        num_leaves=31, objective=None, random_state=0, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1)

In [57]:
Y_pred = lgbm_model.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))

1. The accuracy of the model is 0.4768471229145386

2. Classification report 
              precision    recall  f1-score   support

        0.0       0.52      0.77      0.62      3221
        1.0       0.31      0.13      0.18      2653

avg / total       0.42      0.48      0.42      5874
 

3. Confusion matrix 
 [[2465 2317]
 [ 756  336]] 

4. Roc_Auc score 
 0.411583502235949


## xgboost

In [61]:
import xgboost as xgb 
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [62]:
Y_pred = xgb_model.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))

1. The accuracy of the model is 0.6070820565202588

2. Classification report 
              precision    recall  f1-score   support

        0.0       0.64      0.66      0.65      3221
        1.0       0.57      0.54      0.56      2653

avg / total       0.61      0.61      0.61      5874
 

3. Confusion matrix 
 [[2124 1211]
 [1097 1442]] 

4. Roc_Auc score 
 0.6024108465656892


## Random forest classifier

In [58]:
rf_model = RandomForestClassifier()

rf_model.fit(X_train, Y_train)
Y_pred = rf_model.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))

1. The accuracy of the model is 0.6470888661899897

2. Classification report 
              precision    recall  f1-score   support

        0.0       0.65      0.78      0.71      3221
        1.0       0.64      0.49      0.55      2653

avg / total       0.65      0.65      0.64      5874
 

3. Confusion matrix 
 [[2509 1361]
 [ 712 1292]] 

4. Roc_Auc score 
 0.6465154961395039


In [65]:
 rf_model.feature_importances_

array([ 0.03125631,  0.01737065,  0.02406594,  0.01430159,  0.02582744,
        0.02159597,  0.01802095,  0.02511293,  0.0155039 ,  0.02042773,
        0.01766604,  0.01383374,  0.01567781,  0.01106041,  0.01112234,
        0.012753  ,  0.0138318 ,  0.0102367 ,  0.01375428,  0.01393069,
        0.01175191,  0.01201331,  0.01404701,  0.01287374,  0.01204456,
        0.01272049,  0.01467147,  0.01352093,  0.01354168,  0.01646971,
        0.01328379,  0.01102323,  0.01344159,  0.01280445,  0.01231614,
        0.01238842,  0.01357821,  0.0141944 ,  0.01213573,  0.01397649,
        0.01413593,  0.01565953,  0.01101979,  0.01094694,  0.01079151,
        0.01185763,  0.01213575,  0.01142885,  0.0119439 ,  0.01120112,
        0.01133085,  0.01333845,  0.02408655,  0.03737621,  0.05299594,
        0.        ,  0.02733652,  0.01513035,  0.03067531,  0.00582142,
        0.05063999])

## Gaussian Naive Bayes

In [59]:
from sklearn.naive_bayes import GaussianNB
gb_model = GaussianNB()

gb_model.fit(X_train, Y_train)
Y_pred = gb_model.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))


1. The accuracy of the model is 0.5030643513789581

2. Classification report 
              precision    recall  f1-score   support

        0.0       0.80      0.12      0.22      3221
        1.0       0.48      0.96      0.64      2653

avg / total       0.65      0.50      0.41      5874
 

3. Confusion matrix 
 [[ 401   99]
 [2820 2554]] 

4. Roc_Auc score 
 0.638625604763677


## Support Vector Machine

In [60]:
from sklearn import svm
svm_model = svm.SVC()

svm_model.fit(X_train, Y_train)
Y_pred = svm_model.predict(X_test)

print('1. The accuracy of the model is {}\n'.format(accuracy_score(Y_test, Y_pred)))
print('2. Classification report \n {} \n'.format(classification_report(Y_test, Y_pred)))
print('3. Confusion matrix \n {} \n'.format(confusion_matrix(Y_pred, Y_test)))
print('4. Roc_Auc score \n {}'.format(roc_auc_score(Y_pred, Y_test)))

1. The accuracy of the model is 0.7774940415389854

2. Classification report 
              precision    recall  f1-score   support

        0.0       0.80      0.78      0.79      3221
        1.0       0.75      0.77      0.76      2653

avg / total       0.78      0.78      0.78      5874
 

3. Confusion matrix 
 [[2528  614]
 [ 693 2039]] 

4. Roc_Auc score 
 0.7754613730005695
